In this lab you will do the following steps in order:

1. Load and normalizing the CIFAR10 training and test datasets using
   ``torchvision``
2. Define a Convolution Neural Network
3. Define a loss function and optimizer
4. Train the network on the training data
5. Test the network on the test data

Using ``torchvision``, it’s extremely easy to load CIFAR10.



How to install a different version of a package

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

Use GPU if available

In [ ]:
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
device = torch.device("cuda:0" if train_on_gpu else "cpu")
print(device)

1. Load and normalizing the CIFAR10 training and test datasets using
   ``torchvision``
   
The output of [torchvision datasets](https://pytorch.org/vision/stable/datasets.html#datasets) are PILImage images of range [0, 1].
We [transform](https://pytorch.org/vision/stable/transforms.html) them to normalized Tensors. Then we call the [dataloader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)



In [ ]:
# Define data transformation pipeline
transform = transforms.Compose([
    # Convert PIL images to PyTorch tensors
    transforms.ToTensor(),
    # Normalize pixel values
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the CIFAR-10 training dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

# Create data loader for training data with batch size 4 and shuffling
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

# Load the CIFAR-10 testing dataset
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

# Create data loader for testing data with batch size 1 and shuffling
testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=True, num_workers=2)

# Define class labels for CIFAR-10 dataset
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Let us [show](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html#matplotlib-pyplot-imshow) some of the training images


In [ ]:
import matplotlib.pyplot as plt  # Import library for plotting
import numpy as np  # Import library for numerical computations
from collections import Counter  # Import Counter for counting elements

# Function to display an image
def imshow(image):
    mean=torch.tensor([0.485, 0.456, 0.406])
    std=torch.tensor([0.229, 0.224, 0.225])

    # Unnormalize the image channels to [0, 1]
    image = image.mul(std.unsqueeze(1).unsqueeze(2))  # More efficient element-wise multiplication
    image = image.add(mean.unsqueeze(1).unsqueeze(2))  # Efficient element-wise addition

    image= image.clamp(0, 1)

    # Convert the tensor to a NumPy array
    npimg = image.numpy()
    # Plot the image using matplotlib
    plt.imshow(np.transpose(npimg, (1, 2, 0)))  # Transpose for correct display

# ------------------ Train Loader Section ------------------

print("Train loader:")

# Count the frequency of each class in the training set
stat = dict(Counter(trainset.targets))

# Create a new dictionary with class names as keys
new_stat = stat.copy()
for k in stat.keys():
    new_stat[classes[k]] = stat[k]
    del new_stat[k]

# Print the length of the train loader (number of batches)
print(len(trainloader))

# Print the class distribution in the training set
print(new_stat)

# Get a batch of random training images and their labels
dataiter = iter(trainloader)
images, labels = next(dataiter)

# Print the shape of the image tensor (batch_size, channels, height, width)
print(images.shape)

# Display the images using the imshow function
imshow(torchvision.utils.make_grid(images))

# Print the labels of the images
print(' '.join('%s' % classes[labels[j]] for j in range(4)))  # Print labels for 4 images

# ------------------ Test Loader Section ------------------

print("\nTest loader:")

# Similar steps for the test loader
stat = dict(Counter(testset.targets))
new_stat = stat.copy()
for k in stat.keys():
    new_stat[classes[k]] = stat[k]
    del new_stat[k]

print(len(testloader))
print(new_stat)


2. Define a Convolution Neural Network.
[network layers](https://pytorch.org/docs/stable/nn.html#convolution-layers)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    """
    This class defines a simple convolutional neural network (CNN) architecture
    for image classification.

    Attributes:
        conv1 (nn.Conv2d): First convolutional layer with 3 input channels (RGB),
                           6 output channels, and a kernel size of 5x5.
        pool (nn.MaxPool2d): Max pooling layer with a kernel size of 2x2.
        conv2 (nn.Conv2d): Second convolutional layer with 6 input channels
                           (from the first conv layer), 16 output channels,
                           and a kernel size of 5x5.
        fc1 (nn.Linear): First fully-connected layer that flattens the input
                         from the previous convolutional layers and has 120 neurons.
        fc2 (nn.Linear): Second fully-connected layer with 84 neurons.
        fc3 (nn.Linear): Output layer with 10 neurons, corresponding to the 10 classes
                         in CIFAR-10.

    Methods:
        forward(self, x): Defines the forward pass of the network.
    """

    def __init__(self):
        super(Net, self).__init__()  # Call the superclass constructor
        self.conv1 = nn.Conv2d(3, 6, 5)  # First convolutional layer
        self.pool = nn.MaxPool2d(2, 2)  # Max pooling layer
        self.conv2 = nn.Conv2d(6, 16, 5)  # Second convolutional layer
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # First fully-connected layer
        self.fc2 = nn.Linear(120, 84)  # Second fully-connected layer
        self.fc3 = nn.Linear(84, 10)  # Output layer

    def forward(self, x):
        """
        Defines the forward pass of the neural network.

        Args:
            x (torch.Tensor): Input tensor representing the images.

        Returns:
            torch.Tensor: Output tensor representing the class probabilities.
        """
        x = self.pool(F.relu(self.conv1(x)))  # First convolutional layer with ReLU activation and pooling
        x = self.pool(F.relu(self.conv2(x)))  # Second convolutional layer with ReLU activation and pooling
        # print(x.shape)
        x = x.view(x.shape[0],-1)  # Flatten the output from convolutional layers
        # print(x.shape)
        x = F.relu(self.fc1(x))  # First fully-connected layer with ReLU activation
        x = F.relu(self.fc2(x))  # Second fully-connected layer with ReLU activation
        x = self.fc3(x)  # Output layer
        return x

net = Net()
net.to(device)

Compute the receptive field of the network

In [ ]:
# This line attempts to clone a Git repository using a shell command.
!git clone https://github.com/Fangyh09/pytorch-receptive-field.git

# This line would move the downloaded directory
!mv -v pytorch-receptive-field/torch_receptive_field ./

# Import the 'receptive_field' function from the 'torch_receptive_field' library.
from torch_receptive_field import receptive_field

# Calculate the receptive field of the network 'net' for an input image size of
# 3 channels (RGB) and 32x32 pixels. The 'receptive_field' function would analyze the network architecture and input size to determine
# the receptive field size for each layer and the overall network.
receptive_field(net, input_size=(3, 32, 32))


3. Define a loss function and optimizer

Let's use a Classification [Cross-Entropy](https://pytorch.org/docs/stable/nn.html#loss-functions) loss and [SGD](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD) with momentum.



In [ ]:
import torch.optim as optim  # Import the optim module from PyTorch for optimization algorithms

# Define the loss function
criterion = nn.CrossEntropyLoss()  # Use cross-entropy loss for multi-class classification

# Define the optimizer
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Explanation of the optimizer:
#   - optim.SGD(net.parameters(), lr=0.001, momentum=0.9):
#       - optim.SGD: This selects the Stochastic Gradient Descent (SGD) optimizer.
#       - net.parameters(): This provides the parameters of the network (`net`) to be optimized.
#       - lr=0.001: This sets the learning rate to 0.001 (controls how much the weights are updated).
#       - momentum=0.9: This sets the momentum to 0.9 (a technique to improve convergence).


4. Train the network on the training data


We simply have to loop over our data iterator, and feed the inputs to the
network and optimize.



In [ ]:
num_print_intervals = 4  # Number of times to print statistics

num_print_intervals+=1
print_interval = int(len(trainloader) / num_print_intervals)

# Loop over the dataset multiple times (2 epochs in this case)
for epoch in range(2):
    running_loss=[]  # Initialize a variable to track the total loss for this epoch

    # Iterate over the training data loader
    for i, data in enumerate(trainloader, 0):
        # Get the inputs (images) and labels from the current batch
        inputs, labels = data

        # Move the inputs and labels to the specified device (CPU or GPU)
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Clear the gradients accumulated in the previous iteration
        optimizer.zero_grad()

        # Training loop: forward pass, backward pass, and optimization
        # 1. Forward pass:
        outputs = net(inputs)  # Pass the input images through the network to get predictions (outputs)
        # 2. Calculate loss:
        loss = criterion(outputs, labels)  # Compute the loss based on the predictions (outputs) and ground truth labels
        # 3. Backward pass:
        loss.backward()  # Backpropagate the loss to calculate gradients for each parameter in the network
        # 4. Optimization step:
        optimizer.step()  # Update the weights and biases of the network based on the calculated gradients

        running_loss.append(loss.item())  # Accumulate the loss for this mini-batch
        if i>0 and i % print_interval == 0:  # Check batch interval
            # Print the average loss for the mini-batches
            print('[%d, %5d] loss: %.4f' %
                  (epoch + 1, i + 1, np.mean(running_loss)))
            # Reset the running loss for the next interval
            running_loss=[]

# Training complete
print('Finished Training')


5. Test the network on the test data


We have trained the network for 2 passes over the training dataset.
But we need to check if the network has learnt anything at all.

We will check this by predicting the class label that the neural network
outputs, and checking it against the ground-truth. If the prediction is
correct, we add the sample to the list of correct predictions.

In [ ]:
# Initialize variables to track accuracy
correct = 0
total = 0

# Disable gradient calculation for better performance during evaluation
with torch.no_grad():
    # Loop over the test loader
    for data in testloader:
        # Get the image and label from the current batch
        image, label = data

        # Move the image data to the specified device (CPU or GPU)
        image = image.to(device)

        # Get the network's prediction for the image
        output = net(image)
        # smax = torch.nn.Softmax(dim=1)(output.cpu())

        # Find the class with the highest probability
        _, predicted = torch.max(output.cpu(), 1)  # Equivalent to pred = torch.argmax(output.cpu(), dim=1)

        # Update total number of test images
        total += label.size(0)  # label.size(0) gives the batch size

        # Count correct predictions
        correct += (predicted == label).sum().item()  # Count true positives

# Calculate and print accuracy
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


**!HOMEWORK!**

This homework assignment asks you to performs 2 tasks:

1. Analyze Results with Different Network Parameters:

This involves training the network with various configurations of network parameters and analyzing the impact on performance. Here's a step-by-step approach:

**Choose Network Parameters:**

Select the network parameters you want to experiment with. Common choices include:

Number of layers: You can try increasing or decreasing the number of layers in your chosen network architecture (e.g., convolutional layers in a CNN).
Learning rate: Experiment with different learning rates (e.g., 0.01, 0.001, 0.0001) to find a balance between fast learning and stability.
Other parameters: Depending on your network architecture, there might be additional options like:
Number of filters in convolutional layers: This affects the complexity of features extracted from the data.
Activation functions: Experiment with different activation functions (e.g., ReLU, Leaky ReLU) to introduce non-linearity.
Optimizer parameters: Some optimizers (e.g., Adam) have hyperparameters you can adjust.
Train the network for a different number of epochs.

**Analyze Results:**

Compare the performance of the network across different parameter configurations:

How accuracy/loss changes with different parameter values.
2. Show and Explain Errors of the Best Network:

Once you identify the **best performing network configuration** (based on metrics like accuracy or loss), analyze its errors.
For example you can generate a confusion matrix. This matrix visualizes how often the network predicted each class correctly or incorrectly.

Useful resources:
*   [network layers](https://pytorch.org/docs/stable/nn.html#convolution-layers)
*   [activation function](https://pytorch.org/docs/stable/nn.html#convolution-layers)
*   [loss functions](https://pytorch.org/docs/stable/nn.html#convolution-layers)



